Experimenting with Solr SQL
===========================

This notebook is for experimenting with Solr's Parallel SQL interface, especially via an SQLAlchemy plugin so that it's exactly like normal SQL.

Unfortunatley, current state of play is that Solr 6 does not cope with SQL queries on _aliases_, so it's not much use for text analysis right now.

The SQL system does work okay for Solr 8, but the `SELECT *` logic seems to be a bit brittle (at least via the SQLAlchemy module).  It works pretty reliably if the fields are explicitly enumerated.

In [1]:
!pip install sqlalchemy-solr

In [5]:
from sqlalchemy import create_engine

fc = 'NPLD-FC2017-20190228'

engine = create_engine('solr://solr.api.wa.bl.uk:80/solr/%s' % fc)

# Unfortunately, for Solr 6, we can't query aliases and all fields * leads to class cast exceptions!
# (java.lang.Long cannot be cast to java.lang.String)
rows = engine.execute("SELECT id,url,wayback_date FROM `%s` LIMIT 1" % fc)

for r in rows:
    for column, value in r.items():
        print(column, value)


************************************
Query: SELECT id,url,wayback_date FROM `NPLD-FC2017-20190228` LIMIT 1
************************************
id 20171225120530/r1upsuMttEfpjRI2R4rN7Q==
url http://www.newquayvoice.co.uk/news/5/article/2920/
wayback_date 20171225120530
************************************
Catched StopIteration in fetchone
************************************


In [21]:
#rows = engine.execute("SELECT id,url FROM `NPLD-FC2017-20190228` WHERE (host:'theguardian.com' OR host:'independent.co.uk' OR host:'dailymail.co.uk' OR host:'express.co.uk' OR host:'thesun.co.uk' OR host:'mirror.co.uk' OR host:'dailystar.co.uk') AND ((title:meghan AND title:harry) OR (title:meghan AND title:markle)) LIMIT 1")
rows = engine.execute("SELECT id,url,title,host FROM `NPLD-FC2017-20190228` WHERE host = '(theguardian.com independent.co.uk dailymail.co.uk express.co.uk thesun.co.uk mirror.co.uk dailystar.co.uk)' AND ((title = 'meghan' AND title = 'markle') OR (title = 'meghan' AND title = 'harry')) ORDER BY crawl_date LIMIT 1")

for r in rows:
    for column, value in r.items():
        print(column, value)


************************************
Query: SELECT id,url,title,host FROM `NPLD-FC2017-20190228` WHERE host = '(theguardian.com independent.co.uk dailymail.co.uk express.co.uk thesun.co.uk mirror.co.uk dailystar.co.uk)' AND ((title = 'meghan' AND title = 'markle') OR (title = 'meghan' AND title = 'harry')) ORDER BY crawl_date LIMIT 1
************************************
id 20170101102033/lhGw4C4wmG+0iFG0t7JrFw==
url http://www.dailymail.co.uk/tvshowbiz/article-3976578/amp/Meghan-Markle-enjoys-cocktails-Quantico-star-Priyanka-Chopra-Prince-Harry-tours-Caribbean.html
title Meghan Markle enjoys cocktails with Quantico's Priyanka Chopra while Prince Harry tours
host dailymail.co.uk
************************************
Catched StopIteration in fetchone
************************************


In [27]:
sql_df = pd.read_sql_query(
    "SELECT id,url,title,host,crawl_date FROM `NPLD-FC2017-20190228` WHERE host = '(theguardian.com independent.co.uk dailymail.co.uk express.co.uk thesun.co.uk mirror.co.uk dailystar.co.uk)' AND ((title = 'meghan' AND title = 'markle') OR (title = 'meghan' AND title = 'harry')) ORDER BY crawl_date LIMIT 100000",
    con=engine
)

sql_df

************************************
Query: SELECT id,url,title,host,crawl_date FROM `NPLD-FC2017-20190228` WHERE host = '(theguardian.com independent.co.uk dailymail.co.uk express.co.uk thesun.co.uk mirror.co.uk dailystar.co.uk)' AND ((title = 'meghan' AND title = 'markle') OR (title = 'meghan' AND title = 'harry')) ORDER BY crawl_date LIMIT 100000
************************************


,id,url,title,host,crawl_date
0,20170101102033/lhGw4C4wmG+0iFG0t7JrFw==,http://www.dailymail.co.uk/tvshowbiz/article-3...,Meghan Markle enjoys cocktails with Quantico's...,dailymail.co.uk,2017-01-01 10:20:33
1,20170101102049//M3a8PfNIu5zNH26wfStXA==,http://www.dailymail.co.uk/news/article-392542...,Prince Harry's girlfriend Meghan Markle spotte...,dailymail.co.uk,2017-01-01 10:20:49
2,20170101102057/Erluxdjjqr/jNL4hNzha4A==,http://www.dailymail.co.uk/news/article-402914...,Prince Harry and girlfriend Meghan Markle 'buy...,dailymail.co.uk,2017-01-01 10:20:57
3,20170101102101/z6CUHQC6PS+SWkzOnV6hLw==,http://www.dailymail.co.uk/news/article-402914...,Prince Harry and girlfriend Meghan Markle 'buy...,dailymail.co.uk,2017-01-01 10:21:01
4,20170101102123/Vs9UbyshtSk1D9ZNIyRFCA==,http://www.dailymail.co.uk/news/article-396227...,Prince Harry's girlfriend Meghan Markle says s...,dailymail.co.uk,2017-01-01 10:21:23
...,...,...,...,...,...
21051,20171227215813/rt9ttovXyXjBwGOzBKCWgg==,http://www.mirror.co.uk/news/uk-news/prince-ha...,Prince Harry WON'T take part in the traditiona...,mirror.co.uk,2017-12-27 21:58:13
21052,20171227215854/ZxKG1KVJAy95739ySnIbxg==,http://www.mirror.co.uk/3am/style/celebrity-fa...,"Meghan Markle is Hollywood perfection in £56,0...",mirror.co.uk,2017-12-27 21:58:54
21053,20171227231231/V2VEliskxTDJbIqlCcsF7w==,http://www.mirror.co.uk/3am/celebrity-news/por...,Porn searches for Meghan Markle go through the...,mirror.co.uk,2017-12-27 23:12:31
21054,20171228022515/n/RUSr7G6FbUZuSljxsDew==,http://www.mirror.co.uk/news/uk-news/meghan-ma...,Prince Harry reveals Meghan Markle's first Chr...,mirror.co.uk,2017-12-28 02:25:15


In [1]:
import sys
import pandas as pd
from solrsql import crawl_log_sql, SolrSql, display_enhanced

pd.set_option('display.max_colwidth', -1)

#crawl_log_sql("SELECT * FROM crawl_log_fc WHERE annotations = 'Q:serverMaxSuccessKb' ORDER BY log_timestamp DESC LIMIT 10")
logs = crawl_log_sql()
#print(logs.columns())

#logs.query("SELECT * FROM crawl_log_fc WHERE mimetype LIKE 'image/*' ORDER BY `log_timestamp` DESC LIMIT 100")
df = logs.query("SELECT log_timestamp,source,url,status_code,annotations FROM crawl_log_fc WHERE hop_path = '' AND (NOT annotations = 'WebRenderStatus:200') AND (status_code > -5000 OR status_code < -5000) ORDER BY log_timestamp DESC LIMIT 10")

display_enhanced(df)

************************************
Query: SELECT log_timestamp,source,url,status_code,annotations FROM crawl_log_fc WHERE hop_path = '' AND (NOT annotations = 'WebRenderStatus:200') AND (status_code > -5000 OR status_code < -5000) ORDER BY log_timestamp DESC LIMIT 10
************************************


,log_timestamp,source,url,status_code,annotations,qa_wb_url
0,2021-02-04 18:02:11.621,tid:89483:http://www.tnsglobal.com/,http://www.tnsglobal.com/news-centre/insights,-5002,WebRenderStatus:404 resetQuotas WebRenderCount:1,View in QA Wayback
1,2021-02-04 17:02:37.938,tid:53466:http://www.populus.co.uk/,https://www.populus.co.uk/insights/,-5002,WebRenderStatus:404 resetQuotas WebRenderCount:1,View in QA Wayback
2,2021-02-04 12:00:11.303,tid:67166:https://www.instagram.com/timetochangecampaign/,https://www.instagram.com/timetochangecampaign/,-9998,resetQuotas,View in QA Wayback
3,2021-02-04 11:09:47.311,tid:13213:http://www.theyworkforyou.com/,https://www.theyworkforyou.com/calendar/,-5002,WebRenderStatus:500 resetQuotas WebRenderCount:1,View in QA Wayback
4,2021-02-04 11:03:56.155,tid:31318:http://www.ealingtimes.co.uk/,https://www.ealingtimes.co.uk/education/,-5002,WebRenderStatus:404 resetQuotas WebRenderCount:1,View in QA Wayback
5,2021-02-04 10:49:49.079,tid:122684:https://www.newsnow.co.uk/h/Business+&+Finance/Economy,https://www.newsnow.co.uk/h/Business+&+Finance/Business+%28UK%29,200,resetQuotas WebRenderCount:3,View in QA Wayback
6,2021-02-04 10:46:22.704,tid:108994:https://commonslibrary.parliament.uk/research-briefings/cbp-8867/,http://researchbriefings.files.parliament.uk/documents/CBP-8867/CBP-8867.pdf,301,resetQuotas WebRenderCount:3,View in QA Wayback
7,2021-02-04 10:41:10.869,tid:101370:https://www.mpsontwitter.co.uk,https://www.mpsontwitter.co.uk/list/diff/desc,-5002,WebRenderStatus:404 resetQuotas WebRenderCount:1,View in QA Wayback
8,2021-02-04 10:31:01.966,tid:101370:https://www.mpsontwitter.co.uk,https://www.mpsontwitter.co.uk/list/followers/desc,301,resetQuotas WebRenderCount:3,View in QA Wayback
9,2021-02-04 10:24:13.282,tid:109024:https://www.maths.ox.ac.uk/,https://www.maths.ox.ac.uk/,200,resetQuotas WebRenderCount:3,View in QA Wayback


In [3]:
df = logs.query(
    "SELECT host, source, count(*) as urls \
    FROM crawl_log_fc \
    WHERE annotations = 'Q:serverMaxSuccessKb' \
    GROUP BY host, source \
    ORDER BY urls DESC \
    LIMIT 20"
)

display_enhanced(df)

************************************
Query: SELECT host, source, count(*) as urls     FROM crawl_log_fc     WHERE annotations = 'Q:serverMaxSuccessKb'     GROUP BY host, source     ORDER BY urls DESC     LIMIT 20
************************************


,host,source,urls
0,www.racingpost.com,tid:31637:http://www.racingpost.com/,347990
1,cdn11.bigcommerce.com,tid:129806:https://www.scottsofstow.co.uk/,250865
2,storage.googleapis.com,tid:95604:http://www.ercc.scot/,216056
3,johnlewis.scene7.com,tid:12434:http://www.johnlewis.com/,189677
4,i.dailymail.co.uk,tid:109175:https://www.dailymail.co.uk/news/coronavirus/index.html,135039
5,www.legislation.gov.uk,tid:96456:http://ukscblog.com/,124498
6,i2-prod.grimsbytelegraph.co.uk,tid:31421:http://www.grimsbytelegraph.co.uk/,122552
7,medium.com,tid:100481:https://medium.com/@ncb1947,121702
8,deriv.nls.uk,tid:34936:https://orhighlights.wordpress.com/,119093
9,thumbnailer.mixcloud.com,tid:97162:https://www.mixcloud.com/,92439


In [4]:
tracking = SolrSql('solr://solr8.api.wa.bl.uk:80/solr/tracking')
#print(tracking.columns())

sql_df = tracking.query(
    "SELECT * FROM tracking \
    WHERE kind_s = 'warcs' AND cdx_index_ss = '[* TO *]' AND \
    timestamp_dt = '[NOW/DAY-1DAY TO *]' \
    ORDER BY timestamp_dt ASC \
    LIMIT 10")

sql_df

************************************
Query: SELECT * FROM tracking     WHERE kind_s = 'warcs' AND cdx_index_ss = '[* TO *]' AND     timestamp_dt = '[NOW/DAY-1DAY TO *]'     ORDER BY timestamp_dt ASC     LIMIT 10
************************************


,file_name_s,_root_,hdfs_user_s,stream_s,file_path_s,cdx_records_found_i,year_i,modified_at_dt,recognised_b,file_size_l,...,job_s,kind_s,cdx_index_ss,id,timestamp_dt,file_ext_s,permissions_s,refresh_date_dt,_query_,score
0,BL-NPLD-WEBRENDER-frequent-npld-20210119131039-20210210000349218-04844-n4o6ljbu.warc.gz,,hdfs,frequent,/heritrix/output/frequent-npld/20210119131039/warcs/BL-NPLD-WEBRENDER-frequent-npld-20210119131039-20210210000349218-04844-n4o6ljbu.warc.gz,,2021,2021-02-10 07:15:00,true,1005627565,...,frequent-npld,warcs,[data-heritrix],hdfs://hdfs:54310/heritrix/output/frequent-npld/20210119131039/warcs/BL-NPLD-WEBRENDER-frequent-npld-20210119131039-20210210000349218-04844-n4o6ljbu.warc.gz,2021-02-10 00:03:49.218,.warc.gz,-rw-r--r--,2021-02-11 09:01:53.047,,2
1,BL-NPLD-WEBRENDER-frequent-npld-20210119131039-20210210001256905-04845-n4o6ljbu.warc.gz,,hdfs,frequent,/heritrix/output/frequent-npld/20210119131039/warcs/BL-NPLD-WEBRENDER-frequent-npld-20210119131039-20210210001256905-04845-n4o6ljbu.warc.gz,,2021,2021-02-10 07:12:00,true,1000019747,...,frequent-npld,warcs,[data-heritrix],hdfs://hdfs:54310/heritrix/output/frequent-npld/20210119131039/warcs/BL-NPLD-WEBRENDER-frequent-npld-20210119131039-20210210001256905-04845-n4o6ljbu.warc.gz,2021-02-10 00:12:56.905,.warc.gz,-rw-r--r--,2021-02-11 09:01:53.047,,2
2,BL-NPLD-WEBRENDER-frequent-npld-20210119131039-20210210001438895-04846-n4o6ljbu.warc.gz,,hdfs,frequent,/heritrix/output/frequent-npld/20210119131039/warcs/BL-NPLD-WEBRENDER-frequent-npld-20210119131039-20210210001438895-04846-n4o6ljbu.warc.gz,,2021,2021-02-10 07:30:00,true,1003706090,...,frequent-npld,warcs,[data-heritrix],hdfs://hdfs:54310/heritrix/output/frequent-npld/20210119131039/warcs/BL-NPLD-WEBRENDER-frequent-npld-20210119131039-20210210001438895-04846-n4o6ljbu.warc.gz,2021-02-10 00:14:38.895,.warc.gz,-rw-r--r--,2021-02-11 09:01:53.047,,2
3,BL-NPLD-20210210002056513-11644-75~npld-heritrix3-worker-1~8443.warc.gz,,hdfs,frequent,/heritrix/output/frequent-npld/20210119131039/warcs/BL-NPLD-20210210002056513-11644-75~npld-heritrix3-worker-1~8443.warc.gz,,2021,2021-02-10 09:28:00,true,1006696212,...,frequent-npld,warcs,[data-heritrix],hdfs://hdfs:54310/heritrix/output/frequent-npld/20210119131039/warcs/BL-NPLD-20210210002056513-11644-75~npld-heritrix3-worker-1~8443.warc.gz,2021-02-10 00:20:56.513,.warc.gz,-rw-r--r--,2021-02-11 09:01:53.047,,2
4,BL-NPLD-WEBRENDER-frequent-npld-20210119131039-20210210002126856-04847-n4o6ljbu.warc.gz,,hdfs,frequent,/heritrix/output/frequent-npld/20210119131039/warcs/BL-NPLD-WEBRENDER-frequent-npld-20210119131039-20210210002126856-04847-n4o6ljbu.warc.gz,,2021,2021-02-10 07:26:00,true,1007223457,...,frequent-npld,warcs,[data-heritrix],hdfs://hdfs:54310/heritrix/output/frequent-npld/20210119131039/warcs/BL-NPLD-WEBRENDER-frequent-npld-20210119131039-20210210002126856-04847-n4o6ljbu.warc.gz,2021-02-10 00:21:26.856,.warc.gz,-rw-r--r--,2021-02-11 09:01:53.047,,2
5,BL-NPLD-20210210002312263-11645-75~npld-heritrix3-worker-1~8443.warc.gz,,hdfs,frequent,/heritrix/output/frequent-npld/20210119131039/warcs/BL-NPLD-20210210002312263-11645-75~npld-heritrix3-worker-1~8443.warc.gz,,2021,2021-02-10 08:45:00,true,1013753622,...,frequent-npld,warcs,[data-heritrix],hdfs://hdfs:54310/heritrix/output/frequent-npld/20210119131039/warcs/BL-NPLD-20210210002312263-11645-75~npld-heritrix3-worker-1~8443.warc.gz,2021-02-10 00:23:12.263,.warc.gz,-rw-r--r--,2021-02-11 09:01:53.047,,2
6,BL-NPLD-WEBRENDER-frequent-npld-20210119131039-20210210002854190-04848-n4o6ljbu.warc.gz,,hdfs,frequent,/heritrix/output/frequent-npld/20210119131039/warcs/BL-NPLD-WEBRENDER-frequent-npld-20210119131039-20210210002854190-04848-n4o6ljbu.warc.gz,,2021,2021-02-10 06:43:00,true,1012710912,...,frequent-npld,warcs,[data-heritrix],hdfs://hdfs:54310/heritrix/output/frequent-npld/20210119131039/warcs/BL-NPLD-WEBRENDER-frequent-npld-20210119131039-20210210002854190-04848-n4o6ljbu.warc.gz,2021-02-10 00:28:54.190,.warc.gz,-rw-r--r--,2021-02-11 09:01: